# Black-Scholes-Merton Equation
#### Implementation and exploration of the solutions to the Black-Scholes-Merton for European-style call and put options

The Black-Scholes-Merton equation serves as a fundamental result in financial engineering, the solution of which describes the price of a derivative security as a function of the price of its underlying security and time
$$\frac{\partial f}{\partial t} + rS\frac{\partial f}{\partial S} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2f}{\partial S^2} = rf $$



To give clarity, going forward the following notation will be adopted:
$$K \text{= strike price}$$
$$S \text{= price of the underlying security}$$
$$\sigma \text{= volatility of the underlying security}$$
$$r \text{= risk free rate}$$
$$T \text{= time to maturity}$$

For European-style call and put options,
$$c = S_{0} N(d_{1}) - Ke^{rT}N{d_{2}}$$
$$p = Ke^{-rT}N(-d_{2}) - S_{0}N(-d_{1})$$

where 

$$d_{1} = \frac{\ln(S_{0}/K) + (r + \sigma^2/2 )T}{\sigma \sqrt{T}}  $$

and

$$d_{2} =  \frac{\ln(S_{0}/K) + (r - \sigma^2/2 )T}{\sigma \sqrt{T}} = d_{1} - \sigma T$$

We will now translate the closed form solutions to python code:

In [77]:
import numpy as np
from scipy.stats import norm

# implementation of the solutions to the Black-Scholes-Merton equations as presented in John C. Hull - Options, Futures, and Other Derivatives
# S0 = spot price of underlying security at t0
# K = strike price of european-style option
# T = time to maturity scaled to one year excluding business days (252 days/year)
# r = risk free rate
# vol = volatility of underlying security


# implementation of d1 intermediary calculation
def calculated1(S_0, K, T, r, vol):
    d1 = (np.log(S_0 / K) + (r + (vol**2) / 2) * T) / (vol * np.sqrt(T))
    return d1


# implementation of d2 intermediary calculation
def calculated2(S_0, K, T, r, vol):
    d2 = calculated1(S_0, K, T, r, vol) - vol * np.sqrt(T)
    return d2


# calculation of call price
def calculateCallPrice(S_0, K, T, r, vol):
    d1 = calculated1(S_0, K, T, r, vol)
    d2 = calculated2(S_0, K, T, r, vol)
    ND1 = norm.cdf(d1)  # cdf of normal distribution from scipy
    ND2 = norm.cdf(d2)  # cdf of normal distribution from scipy
    price = S_0 * ND1 - K * np.exp(-r * T) * ND2  # solution per Hull
    return price


# calculation of put price
def calculatePutPrice(S_0, K, T, r, vol):
    d1 = calculated1(S_0, K, T, r, vol)
    d2 = calculated2(S_0, K, T, r, vol)
    ND1 = norm.cdf(-d1)  # cdf of normal distribution from scipy
    ND2 = norm.cdf(-d2)  # cdf of normal distribution from scipy
    price = K * np.exp(-r * T) * ND2 - S_0 * ND1  # solution per Hull
    return price

Performing a simple test of the implementation:

In [78]:
S_0 = 50
K = 50
T = 0.5
r = 0.05
vol = 0.2
print(f"d1: {calculated1(S_0, K, T, r, vol)}")
print(f"d2: {calculated2(S_0, K, T, r, vol)}")
print(f"Call price: {calculateCallPrice(S_0, K, T, r, vol)}")
print(f"Put price: {calculatePutPrice(S_0, K, T, r, vol)}")


Calculated d1: 0.24748737341529162
Calculated d2: 0.10606601717798209
Call price: 3.444364288840312
Put price: 2.20985989025694


These figures may be verified by manually performing the calculations.

Next, we generate data to explore our solutions graphically.

In [79]:
#setting variables
strikePrice = 20
expiryDate = 504
maxPrice = 40
volatility = 0.19
rate = 0.0451
resolution = 100

#ticks represents the time mesh
ticks = np.linspace(0, expiryDate - 1, resolution)
#generating a range of underlying stock price values
S0 = np.linspace(0.1, maxPrice, resolution)
#generating an array T which represents a decreasing time to maturity
expDate = expiryDate * np.ones(resolution)
T = (expDate - ticks) / 252
#generating a meshgrid
T_grid, S0_grid = np.meshgrid(T, S0)

#initializing price arrays
callPrices = np.zeros_like(T_grid)
putPrices = np.zeros_like(T_grid)

#populating the arrays with output from functions created earlier
for i in range(len(S0)):
    callPrices[i, :] = calculateCallPrice(
        S0_grid[i, :], strikePrice, T_grid[i, :], rate, volatility
    )
    putPrices[i, :] = calculatePutPrice(
        S0_grid[i, :], strikePrice, T_grid[i, :], rate, volatility
    )

Finally, we set up a graph showing the options prices over time.

In [80]:
import plotly.graph_objects as go
import nbformat
fig = go.Figure()

fig.add_trace(
    go.Surface(
        x=T_grid, y=S0_grid, z=callPrices, colorscale="Viridis", name="Call Prices"
    )
)

fig.add_trace(
    go.Surface(
        x=T_grid,
        y=S0_grid,
        z=putPrices,
        colorscale="Cividis",
        name="Put Prices",
        showscale=False,
        opacity=0.8,
    )
)

fig.update_layout(
    title="Call and Put Prices",
    scene=dict(
        xaxis_title="Time to Expiry",
        yaxis_title="Initial Stock Price",
        zaxis_title="Option Price",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
)



Reference: John C. Hull - Options, Futures, and Other Derivatives